In [5]:
# !pip install --upgrade pip
# !pip install numpy pandas rectools 'rectools[lightfm]' seaborn

# ⬇️ IMPORTS

In [6]:
# System Imports
import warnings
warnings.filterwarnings("ignore")

# Typings
from typing import List, Dict, Tuple

# Datetime
from datetime import datetime as dt

# Data Processing
import numpy as np
import pandas as pd

# Models
import lightfm
from lightfm import LightFM
from implicit.als import AlternatingLeastSquares

# RecTools
from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import (
    Precision, Recall,
    MAP, MRR,
    MeanInvUserFreq, Serendipity,
    CatalogCoverage, NDCG, Precision,
    AvgRecPopularity, CoveredUsers,
    calc_metrics,
)

from rectools.models import (
    PureSVDModel,
    ImplicitALSWrapperModel,
    LightFMWrapperModel,
    PopularModel,
    RandomModel,
    model_from_config
)

# Visualizations
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

# 🌆 Environment

In [7]:
TOP_K = 10
NUM_THREADS = 6
RANDOM_STATE = 23

# Randomization
np.random.seed(RANDOM_STATE)

# Functions

In [8]:
def validate_models(models, dataset, train, test, catalog, metrics, hot_users):
    results = []
    for model_name, model in tqdm_notebook(models.items()):
        model_quality = {'model': model_name}

        model.fit(dataset)
        recos = model.recommend(
            k=TOP_K,
            dataset=dataset,
            users=hot_users,
            filter_viewed=True,
        )

        metric_values = calc_metrics(
            metrics, recos, test, train, catalog=catalog
        )

        model_quality.update(metric_values)
        results.append(model_quality)

    df_quality = pd.DataFrame(results).T
    df_quality.columns = df_quality.iloc[0]
    df_quality.drop('model', inplace=True)
    return df_quality

# 💎 Данные

In [11]:
df = pd.read_csv('../data/debank_data.csv')
df.drop(columns=["Unnamed: 0"], inplace=True)
df.dropna(inplace=True)

# Basic statistics
print("Basic Statistics:")
print(df.shape)
print(f"Number of unique users: {df['user'].nunique():,}")
print(f"Number of unique pools: {df['pool'].nunique():,}")
print(f"Number of unique types: {df['types'].nunique():,}")
print(f"Number of unique chains: {df['chain'].nunique():,}")
print(f"Number of unique protocols: {df['protocol'].nunique():,}")

df.head()

Basic Statistics:
(217256, 7)
Number of unique users: 19,378
Number of unique pools: 17,032
Number of unique types: 9
Number of unique chains: 54
Number of unique protocols: 2,057


,user,protocol,chain,pool,balance,types,balance_share
0,0xfd09f0296af88ac777c137ecd92d85583a9b9e4a,ftm_pwawallet,ftm,FTM,1.053008e+06,common,1.000000
1,0x637b935cba030aeb876eae07aa7ff637166de4d6,avax_unifiprotocol,avax,UNFI/AVAX,1.319143e+00,common,0.000002
2,0x637b935cba030aeb876eae07aa7ff637166de4d6,balancer,eth,DAI/YFID,8.536821e+01,common,0.000111
3,0x637b935cba030aeb876eae07aa7ff637166de4d6,bsc_acryptos,bsc,ACSI,3.888207e+02,common,0.000504
4,0x637b935cba030aeb876eae07aa7ff637166de4d6,bsc_acryptos,bsc,ACS/WAV/BNB,3.686864e+03,common,0.004783


# 🔭 RecTools

# 🔄 Предобработка данных

In [12]:
# Coluns
USER_COLUMNS = ["user"]
ITEM_COLUMNS = ["protocol", "chain", "pool", "types"]
FINAL_COLUMNS = ["user_id", "item_id", "datetime", "weight"]

# Prepare dataframe
df["datetime"] = pd.to_datetime(dt.now())
df.rename(columns={"balance_share": "weight"}, inplace=True)
df["item"] = df[ITEM_COLUMNS].apply(lambda x: '_'.join(x), axis=1)

# Get Unique values
uniq_users = df['user'].unique().tolist()
uniq_items = df["item"].unique().tolist()

# Create Mappings
ID_TO_USER = dict(enumerate(uniq_users))
ID_TO_ITEM = dict(enumerate(uniq_items))
USER_TO_ID = {user: idx for idx, user in ID_TO_USER.items()}
ITEM_TO_ID = {item: idx for idx, item in ID_TO_ITEM.items()}

# Map columns
df["user_id"] = df["user"].map(USER_TO_ID)
df["item_id"] = df["item"].map(ITEM_TO_ID)
df_reco = df[FINAL_COLUMNS].copy()

In [13]:
print(df["user"].nunique())
print(df["item"].nunique())

19378
28909


# ✂️ Splitter

In [14]:
np.random.seed(RANDOM_STATE)
test_size = df_reco.shape[0] // 4

train = df_reco.__deepcopy__()
test = df_reco.sample(test_size)
train.drop(test.index, inplace=True)
dataset = Dataset.construct(train)

In [15]:
catalog = train[Columns.Item].unique()
test_users = test[Columns.User].unique()
cold_users = set(test_users) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
hot_users = test[Columns.User].unique()

# 🤖 Models

In [16]:
metrics_name = {
    'Precision': Precision(TOP_K),
    'Recall': Recall(TOP_K),
    'MAP': MAP(TOP_K),
    'MRR': MRR(TOP_K),
    "NDCG": NDCG(TOP_K, divide_by_achievable=True),  # ранжирование
    "Новизна": MeanInvUserFreq(TOP_K),  # новизна
    "Popularity Bias": AvgRecPopularity(TOP_K),  # popularity bias
    "serendipity": Serendipity(TOP_K),  # WOW effect
    "catalog_coverage": CatalogCoverage(TOP_K),  # AggregateDiversity
    "covered_users": CoveredUsers(TOP_K),  # Data Quality
    # "intersections": Intersection(TOP_K),  # Пересечение рекомендаций
}

metrics = {}
for metric_name, metric in metrics_name.items():
    metrics[f'{metric_name}@{TOP_K}'] = metric

## Popular Model

In [295]:
popular = (
    df_reco
    .groupby(["item_id"])
    .agg({
        "user_id": "nunique",
        "datetime": "count",
    })
    .rename(columns={
        "user_id": "au",
        "datetime": "cnt"
    })
    .sort_values("au", ascending=False)
    .reset_index()
)[:TOP_K]["item_id"].tolist()

In [296]:
[ID_TO_ITEM[item_id] for item_id in popular]

['lido_eth_ETH_common',
 'arb_gmx_arb_GMX_common',
 'arb_gmx_arb_WBTC/WETH/USDC/LINK/UNI/USDT/MIM/FRAX/DAI_common',
 'arb_gmx_arb_esGMX_common',
 'avax_wonderland_avax_TIME_common',
 'aave2_eth_AAVE_common',
 'bsc_pancakeswap_bsc_Cake_common',
 'op_synthetix_op_SNX_lending',
 'looksrare_eth_LOOKS_common',
 'matic_quickswap_matic_QUICK_common']

In [322]:
models = {
    "Popular": PopularModel(),
    "Random": RandomModel(random_state=RANDOM_STATE)
}
df_quality = validate_models(models, dataset, train, test, catalog, metrics, hot_users)
df_quality.style.highlight_max(color='lightgreen', axis=1)

  0%|          | 0/2 [00:00<?, ?it/s]

model,Popular,Random
Precision@10,0.025587,0.000111
Recall@10,0.067622,0.000146
NDCG@10,0.062751,0.000125
MRR@10,0.109638,0.000278
MAP@10,0.037426,0.000025
Новизна@10,4.646647,13.097313
catalog_coverage@10,23,24244
Popularity Bias@10,837.909903,6.414097
serendipity@10,0.000004,0.000001
covered_users@10,1.000000,1.000000


## SVD

In [327]:
FACTORS = np.arange(70, 300)

# Models Create
models = {}
for factors in FACTORS:
    models[f"SVD_rs_{factors}"] = PureSVDModel(
        maxiter=25,
        factors=factors,
        random_state=RANDOM_STATE,
    )

In [328]:
df_quality = validate_models(models, dataset, train, test, catalog, metrics, hot_users)
df_quality.style.highlight_max(color='lightgreen', axis=1)

  0%|          | 0/230 [00:00<?, ?it/s]

model,SVD_rs_70,SVD_rs_71,SVD_rs_72,SVD_rs_73,SVD_rs_74,SVD_rs_75,SVD_rs_76,SVD_rs_77,SVD_rs_78,SVD_rs_79,SVD_rs_80,SVD_rs_81,SVD_rs_82,SVD_rs_83,SVD_rs_84,SVD_rs_85,SVD_rs_86,SVD_rs_87,SVD_rs_88,SVD_rs_89,SVD_rs_90,SVD_rs_91,SVD_rs_92,SVD_rs_93,SVD_rs_94,SVD_rs_95,SVD_rs_96,SVD_rs_97,SVD_rs_98,SVD_rs_99,SVD_rs_100,SVD_rs_101,SVD_rs_102,SVD_rs_103,SVD_rs_104,SVD_rs_105,SVD_rs_106,SVD_rs_107,SVD_rs_108,SVD_rs_109,SVD_rs_110,SVD_rs_111,SVD_rs_112,SVD_rs_113,SVD_rs_114,SVD_rs_115,SVD_rs_116,SVD_rs_117,SVD_rs_118,SVD_rs_119,SVD_rs_120,SVD_rs_121,SVD_rs_122,SVD_rs_123,SVD_rs_124,SVD_rs_125,SVD_rs_126,SVD_rs_127,SVD_rs_128,SVD_rs_129,SVD_rs_130,SVD_rs_131,SVD_rs_132,SVD_rs_133,SVD_rs_134,SVD_rs_135,SVD_rs_136,SVD_rs_137,SVD_rs_138,SVD_rs_139,SVD_rs_140,SVD_rs_141,SVD_rs_142,SVD_rs_143,SVD_rs_144,SVD_rs_145,SVD_rs_146,SVD_rs_147,SVD_rs_148,SVD_rs_149,SVD_rs_150,SVD_rs_151,SVD_rs_152,SVD_rs_153,SVD_rs_154,SVD_rs_155,SVD_rs_156,SVD_rs_157,SVD_rs_158,SVD_rs_159,SVD_rs_160,SVD_rs_161,SVD_rs_162,SVD_rs_163,SVD_rs_164,SVD_rs_165,SVD_rs_166,SVD_rs_167,SVD_rs_168,SVD_rs_169,SVD_rs_170,SVD_rs_171,SVD_rs_172,SVD_rs_173,SVD_rs_174,SVD_rs_175,SVD_rs_176,SVD_rs_177,SVD_rs_178,SVD_rs_179,SVD_rs_180,SVD_rs_181,SVD_rs_182,SVD_rs_183,SVD_rs_184,SVD_rs_185,SVD_rs_186,SVD_rs_187,SVD_rs_188,SVD_rs_189,SVD_rs_190,SVD_rs_191,SVD_rs_192,SVD_rs_193,SVD_rs_194,SVD_rs_195,SVD_rs_196,SVD_rs_197,SVD_rs_198,SVD_rs_199,SVD_rs_200,SVD_rs_201,SVD_rs_202,SVD_rs_203,SVD_rs_204,SVD_rs_205,SVD_rs_206,SVD_rs_207,SVD_rs_208,SVD_rs_209,SVD_rs_210,SVD_rs_211,SVD_rs_212,SVD_rs_213,SVD_rs_214,SVD_rs_215,SVD_rs_216,SVD_rs_217,SVD_rs_218,SVD_rs_219,SVD_rs_220,SVD_rs_221,SVD_rs_222,SVD_rs_223,SVD_rs_224,SVD_rs_225,SVD_rs_226,SVD_rs_227,SVD_rs_228,SVD_rs_229,SVD_rs_230,SVD_rs_231,SVD_rs_232,SVD_rs_233,SVD_rs_234,SVD_rs_235,SVD_rs_236,SVD_rs_237,SVD_rs_238,SVD_rs_239,SVD_rs_240,SVD_rs_241,SVD_rs_242,SVD_rs_243,SVD_rs_244,SVD_rs_245,SVD_rs_246,SVD_rs_247,SVD_rs_248,SVD_rs_249,SVD_rs_250,SVD_rs_251,SVD_rs_252,SVD_rs_253,SVD_rs_254,SVD_rs_255,SVD_rs_256,SVD_rs_257,SVD_rs_258,SVD_rs_259,SVD_rs_260,SVD_rs_261,SVD_rs_262,SVD_rs_263,SVD_rs_264,SVD_rs_265,SVD_rs_266,SVD_rs_267,SVD_rs_268,SVD_rs_269,SVD_rs_270,SVD_rs_271,SVD_rs_272,SVD_rs_273,SVD_rs_274,SVD_rs_275,SVD_rs_276,SVD_rs_277,SVD_rs_278,SVD_rs_279,SVD_rs_280,SVD_rs_281,SVD_rs_282,SVD_rs_283,SVD_rs_284,SVD_rs_285,SVD_rs_286,SVD_rs_287,SVD_rs_288,SVD_rs_289,SVD_rs_290,SVD_rs_291,SVD_rs_292,SVD_rs_293,SVD_rs_294,SVD_rs_295,SVD_rs_296,SVD_rs_297,SVD_rs_298,SVD_rs_299
Precision@10,0.020473,0.020241,0.020288,0.020872,0.020900,0.020947,0.021012,0.021030,0.020993,0.021123,0.020937,0.020807,0.021513,0.022246,0.022292,0.022097,0.022218,0.021958,0.021958,0.022079,0.021884,0.021782,0.021875,0.021606,0.021541,0.021800,0.021828,0.022246,0.022292,0.022599,0.022664,0.022877,0.023109,0.023081,0.022766,0.022571,0.022459,0.022552,0.022394,0.022367,0.022274,0.022432,0.021912,0.021745,0.021912,0.021606,0.021420,0.021476,0.021012,0.020854,0.021002,0.021095,0.021086,0.021095,0.021132,0.021049,0.020724,0.020687,0.020640,0.020677,0.020445,0.020445,0.020575,0.020640,0.020510,0.020464,0.020483,0.020251,0.020436,0.020297,0.020121,0.020640,0.020826,0.020640,0.020436,0.021095,0.021244,0.021216,0.021271,0.021457,0.021494,0.021346,0.021318,0.021216,0.021160,0.021142,0.021281,0.021448,0.021513,0.021485,0.021476,0.021392,0.021411,0.021550,0.021615,0.021633,0.021643,0.021596,0.021745,0.021448,0.021411,0.021401,0.021513,0.021476,0.021485,0.021494,0.021503,0.021531,0.021568,0.021392,0.021401,0.021374,0.021169,0.021021,0.020956,0.020937,0.020984,0.021030,0.021114,0.021058,0.021123,0.021030,0.021318,0.021392,0.021383,0.021466,0.021374,0.021197,0.021206,0.021151,0.021346,0.021355,0.022162,0.022302,0.022339,0.022283,0.022218,0.022172,0.022246,0.022329,0.022320,0.022329,0.022320,0.022329,0.022255,0.022107,0.022097,0.022032,0.021819,0.021838,0.021828,0.021838,0.021977,0.021717,0.021763,0.021652,0.021782,0.022357,0.022432,0.022497,0.022385,0.022432,0.022571,0.022524,0.022

In [329]:
models['SVD_rs_103'].get_config()

{'cls': rectools.models.pure_svd.PureSVDModel,
 'verbose': 0,
 'factors': 103,
 'tol': 0.0,
 'maxiter': 25,
 'random_state': 23,
 'use_gpu': False,
 'recommend_n_threads': 0,
 'recommend_use_gpu_ranking': True}

In [330]:
svd_config = {
    'cls': 'PureSVDModel',
    'factors': 103,
    'maxiter': 25,
    'random_state': 23,
    'recommend_n_threads': 0
}

## ALS

In [ ]:
models = {}

# Parameters Grid - ALS
N_FACTORS = np.arange(1, 20, 2)
REGULARIZATIONS = np.arange(1, 1, 0.2)

# Create ALS Models
for regularization in REGULARIZATIONS:
    for n_factors in N_FACTORS:
        models[f"iALS_factors={n_factors}_reg={regularization}"] = ImplicitALSWrapperModel(
            model=AlternatingLeastSquares(
                factors=n_factors,
                regularization=regularization,
                random_state=RANDOM_STATE,
                num_threads=NUM_THREADS
            )
        )

In [ ]:
df_quality = validate_models(models, dataset, train, test, catalog, metrics, hot_users)
df_quality.style.highlight_max(color='lightgreen', axis=1)

In [303]:
models['iALS_factors=18_reg=0.2'].get_config()

{'cls': rectools.models.implicit_als.ImplicitALSWrapperModel,
 'verbose': 0,
 'model': {'cls': 'AlternatingLeastSquares',
  'factors': 18,
  'regularization': 0.2,
  'alpha': 1.0,
  'dtype': dtype('float32'),
  'use_native': True,
  'use_cg': True,
  'use_gpu': False,
  'iterations': 15,
  'calculate_training_loss': False,
  'num_threads': 6,
  'random_state': 23},
 'fit_features_together': False,
 'recommend_n_threads': None,
 'recommend_use_gpu_ranking': None}

In [331]:
als_config = {
    'cls': 'ImplicitALSWrapperModel',
    'verbose': 0,
    'model': {
        'cls': 'AlternatingLeastSquares',
        'factors': 18,
        'regularization': 0.2,
        'alpha': 1.0,
        'use_native': True,
        'use_cg': True,
        'use_gpu': False,
        'iterations': 15,
        'calculate_training_loss': False,
        'num_threads': 6,
        'random_state': 23
    },
    'fit_features_together': False,
    'recommend_n_threads': None,
    'recommend_use_gpu_ranking': None
}

## Light FM

In [309]:
N_EPOCHS = 1 # Lightfm
USER_ALPHA = 0 # Lightfm
ITEM_ALPHA = 0 # Lightfm
LEARNING_RATE = 0.05 # Lightfm

In [310]:
models = {}

# Parameters Grid
N_FACTORS = np.arange(1, 20, 1)

# Models Create
lightfm_losses = ('logistic', 'bpr', 'warp')

for loss in lightfm_losses:
    for n_factors in N_FACTORS:
        models[f"LightFM_{loss}_{n_factors}"] = LightFMWrapperModel(
            LightFM(
                no_components=n_factors,
                loss=loss,
                random_state=RANDOM_STATE,
                learning_rate=LEARNING_RATE,
                user_alpha=USER_ALPHA,
                item_alpha=ITEM_ALPHA,
            ),
            epochs=N_EPOCHS,
            num_threads=NUM_THREADS,
        )

In [ ]:
df_quality = validate_models(models, dataset, train, test, catalog, metrics, hot_users)
df_quality.style.highlight_max(color='lightgreen', axis=1)

  0%|          | 0/57 [00:00<?, ?it/s]

model,LightFM_logistic_1,LightFM_logistic_2,LightFM_logistic_3,LightFM_logistic_4,LightFM_logistic_5,LightFM_logistic_6,LightFM_logistic_7,LightFM_logistic_8,LightFM_logistic_9,LightFM_logistic_10,LightFM_logistic_11,LightFM_logistic_12,LightFM_logistic_13,LightFM_logistic_14,LightFM_logistic_15,LightFM_logistic_16,LightFM_logistic_17,LightFM_logistic_18,LightFM_logistic_19,LightFM_bpr_1,LightFM_bpr_2,LightFM_bpr_3,LightFM_bpr_4,LightFM_bpr_5,LightFM_bpr_6,LightFM_bpr_7,LightFM_bpr_8,LightFM_bpr_9,LightFM_bpr_10,LightFM_bpr_11,LightFM_bpr_12,LightFM_bpr_13,LightFM_bpr_14,LightFM_bpr_15,LightFM_bpr_16,LightFM_bpr_17,LightFM_bpr_18,LightFM_bpr_19,LightFM_warp_1,LightFM_warp_2,LightFM_warp_3,LightFM_warp_4,LightFM_warp_5,LightFM_warp_6,LightFM_warp_7,LightFM_warp_8,LightFM_warp_9,LightFM_warp_10,LightFM_warp_11,LightFM_warp_12,LightFM_warp_13,LightFM_warp_14,LightFM_warp_15,LightFM_warp_16,LightFM_warp_17,LightFM_warp_18,LightFM_warp_19
Precision@10,0.014951,0.015870,0.016139,0.016093,0.016093,0.016111,0.016037,0.016037,0.016046,0.016019,0.016028,0.016028,0.016009,0.016028,0.016009,0.016000,0.016102,0.016000,0.015991,0.001995,0.001968,0.001735,0.001735,0.002070,0.001318,0.002218,0.001968,0.001958,0.001624,0.001754,0.001940,0.001652,0.001643,0.001735,0.001921,0.001893,0.001763,0.001735,0.014265,0.014070,0.015480,0.014858,0.015870,0.015731,0.012872,0.014200,0.016381,0.014459,0.014626,0.014190,0.013661,0.012752,0.014478,0.014367,0.015601,0.014988,0.014348
Recall@10,0.046528,0.048552,0.049342,0.049072,0.048899,0.049090,0.048750,0.048652,0.048731,0.048820,0.048582,0.048567,0.048550,0.048526,0.048621,0.048640,0.048787,0.048498,0.048485,0.009075,0.009297,0.007844,0.007256,0.008743,0.006421,0.009611,0.009219,0.009092,0.007942,0.007911,0.009141,0.007783,0.007924,0.007359,0.008382,0.008074,0.008065,0.007983,0.044308,0.043409,0.046120,0.046753,0.047356,0.047089,0.042243,0.045154,0.049498,0.045302,0.045060,0.044072,0.044061,0.040876,0.045789,0.044818,0.047545,0.047359,0.044958
NDCG@10,0.044825,0.046706,0.047000,0.047000,0.046639,0.046969,0.046585,0.046880,0.046976,0.047028,0.046580,0.046465,0.046521,0.046256,0.046686,0.046467,0.046671,0.046508,0.046800,0.005977,0.005700,0.004803,0.005182,0.005151,0.004973,0.006261,0.004981,0.005985,0.004230,0.005469,0.005964,0.005591,0.005788,0.005332,0.005938,0.005841,0.005759,0.005471,0.042234,0.043886,0.046147,0.044717,0.047074,0.045530,0.041756,0.045984,0.048087,0.043747,0.045337,0.043137,0.042460,0.042907,0.043755,0.045304,0.048144,0.046139,0.044070
MRR@10,0.080185,0.083635,0.083219,0.083516,0.082786,0.083546,0.082721,0.083419,0.083672,0.083746,0.082877,0.082492,0.082723,0.082087,0.082974,0.082471,0.082906,0.082725,0.083450,0.007446,0.006450,0.005376,0.006659,0.007107,0.006900,0.007711,0.005150,0.007533,0.004238,0.006905,0.007320,0.007410,0.007340,0.007067,0.007419,0.007805,0.007650,0.006991,0.075382,0.080620,0.085432,0.079240,0.085145,0.081795,0.075388,0.085478,0.086543,0.077735,0.083273,0.077514,0.075374,0.081785,0.077154,0.083759,0.090741,0.085045,0.079672
MAP@10,0.029174,0.030081,0.030154,0.030197,0.029929,0.030131,0.029924,0.030165,0.030216,0.030257,0.029928,0.029861,0.029896,0.029690,0.030040,0.029868,0.029951,0.029893,0.030105,0.003782,0.003502,0.002866,0.003401,0.002577,0.003550,0.003882,0.002607,0.003798,0.002267,0.003560,0.003870,0.003781,0.004175,0.003541,0.003944,0.003821,0.003822,0.003618,0.027362,0.028990,0.029771,0.029118,0.030464,0.029300,0.028055,0.030465,0.030764,0.028773,0.029923,0.028385,0.028153,0.028828,0.028807,0.029805,0.031083,0.030052,0.028861
Новизна@10,5.852492,5.717123,5.693252,5.702602,5.711924,5.709726,5.721549,5.691434,5.721059,5.724911,5.719498,5.720113,5.721470,5.723784,5.723510,5.723016,5.673854,5.725609,5.725829,7.350747,7.295533,7.393408,7.310427,7.195565,7.486182,7.166442,7.293916,7.245646,7.377163,7.363678,7.241031,7.414882,7.415177,7.261603,7.223084,7.354098,7.392417,7.355235,6.052649,5.921270,5.872966,5.918738,5.886023,5.918433,6.119249,6.117826,5.736356,5.887859

In [316]:
models['LightFM_warp_17'].get_config()

{'cls': rectools.models.lightfm.LightFMWrapperModel,
 'verbose': 0,
 'model': {'cls': lightfm.lightfm.LightFM,
  'no_components': 17,
  'k': 5,
  'n': 10,
  'learning_schedule': 'adagrad',
  'loss': 'warp',
  'learning_rate': 0.05,
  'rho': 0.95,
  'epsilon': 1e-06,
  'item_alpha': 0.0,
  'user_alpha': 0.0,
  'max_sampled': 10,
  'random_state': 23},
 'epochs': 1,
 'num_threads': 6,
 'recommend_n_threads': None,
 'recommend_use_gpu_ranking': True}

In [334]:
lightfm_config = {
    'cls': 'LightFMWrapperModel',
    'verbose': 0,
    'model': {
        'cls': lightfm.lightfm.LightFM,
        'no_components': 17,
        'k': 5,
        'n': 10,
        'learning_schedule': 'adagrad',
        'loss': 'warp',
        'learning_rate': 0.05,
        'rho': 0.95,
        'epsilon': 1e-06,
        'item_alpha': 0.0,
        'user_alpha': 0.0,
        'max_sampled': 10,
        'random_state': 23
    },
    'epochs': 1,
    'num_threads': 6,
    'recommend_n_threads': None,
    'recommend_use_gpu_ranking': True
}

## Сравнение лучших моделей

In [339]:
models = {
    "Random": RandomModel(random_state=RANDOM_STATE),
    "Popular": PopularModel(),
    "SVD": model_from_config(svd_config),
    "ALS": model_from_config(als_config),
    "LightFM": model_from_config(lightfm_config),
}

In [340]:
df_quality = validate_models(models, dataset, train, test, catalog, metrics, hot_users)
df_quality.style.highlight_max(color='lightgreen', axis=1)

  0%|          | 0/5 [00:00<?, ?it/s]

model,Random,Popular,SVD,ALS,LightFM
Precision@10,0.000111,0.025587,0.023081,0.015053,0.015601
Recall@10,0.000146,0.067622,0.090479,0.052816,0.047545
NDCG@10,0.000125,0.062751,0.076737,0.040865,0.048144
MRR@10,0.000278,0.109638,0.108694,0.058404,0.090741
MAP@10,0.000025,0.037426,0.054366,0.025834,0.031083
Новизна@10,13.097313,4.646647,7.654605,6.633223,5.969021
catalog_coverage@10,24244,23,928,141,18
Popularity Bias@10,6.414097,837.909903,219.825123,336.681151,526.889476
serendipity@10,0.000001,0.000004,0.000090,0.000027,0.000005
covered_users@10,1.000000,1.000000,1.000000,1.000000,1.000000


# Черновик

## Word2Vec

In [ ]:
# target = 'pool'
# data = df.groupby('user').agg({target: set}).reset_index()
# data['pool'].apply(lambda tokens: [token.split('/') for token in tokens])

## KNN

In [ ]:
def get_mapping(train_df, col):
    inv_mapping = dict(enumerate(train_df[col].unique()))
    mapping = {v: k for k, v in inv_mapping.items()}
    return inv_mapping, mapping

In [ ]:
# Encode users and their item combinations
user_ids, user_uniqs = pd.factorize(df["user"])
item_ids, item_uniqs = pd.factorize(list(zip(
    df['protocol'], df['chain'], df['pool'], df['types']
)))

# Create new columns
rdf = df.__deepcopy__()
rdf['user_id'] = user_ids
rdf['item_id'] = item_ids
rdf['datetime'] = pd.to_datetime(dt.now())
rdf = rdf[['user_id','item_id', 'balance_share']].copy()
rdf.rename(columns={'balance_share': 'weight'}, inplace=True)